### Import Dependencies

In [1]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchText, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import add_to_shopping_cart, get_shopping_cart, remove_from_shopping_cart, get_formatted_items_context, get_formatted_reviews_context

from langgraph.checkpoint.postgres import PostgresSaver
import psycopg2
from psycopg2.extras import RealDictCursor, execute_batch
import numpy as np

### Fictional Warehouses

In [2]:
warehouses = [
    {
        "warehouse_id": "DE-BER-01",
        "warehouse_location": "Berlin, Germany",
        "warehouse_name": "Berlin Distribution Center"
    },
    {
        "warehouse_id": "DE-MUN-01",
        "warehouse_location": "Munich, Germany",
        "warehouse_name": "Munich Logistics Hub"
    },
    {
        "warehouse_id": "DE-HAM-01",
        "warehouse_location": "Hamburg, Germany",
        "warehouse_name": "Hamburg North Warehouse"
    },
    {
        "warehouse_id": "FR-PAR-01",
        "warehouse_location": "Paris, France",
        "warehouse_name": "Paris Central Depot"
    },
    {
        "warehouse_id": "FR-LYO-01",
        "warehouse_location": "Lyon, France",
        "warehouse_name": "Lyon Regional Warehouse"
    },
    {
        "warehouse_id": "FR-MAR-01",
        "warehouse_location": "Marseille, France",
        "warehouse_name": "Marseille Mediterranean Hub"
    }
]

### Simulate Stock Availability for each of the warehouse

#### Retrieve all item IDs from Amazon items Qdrant Collection

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [4]:
dummy_vector = np.zeros(1536).tolist()

In [5]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-01-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [6]:
payload.points

[ScoredPoint(id=379, version=3, score=0.0, payload={'parent_asin': 'B0C2YMY2QB'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=452, version=3, score=0.0, payload={'parent_asin': 'B09PYN3P4B'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=454, version=3, score=0.0, payload={'parent_asin': 'B0BM79FR3B'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=201, version=3, score=0.0, payload={'parent_asin': 'B0BN54T6XD'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=328, version=3, score=0.0, payload={'parent_asin': 'B0BBM919DP'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=293, version=3, score=0.0, payload={'parent_asin': 'B0BYSY8NXV'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=367, version=3, score=0.0, payload={'parent_asin': 'B09F76BCZS'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=443, version=3, score=0.0, payload={'parent_asin': 'B0BQ6P583M'}, vector=N

In [7]:
parent_asin_list = [point.payload["parent_asin"] for point in payload.points]

In [8]:
parent_asin_list

['B0C2YMY2QB',
 'B09PYN3P4B',
 'B0BM79FR3B',
 'B0BN54T6XD',
 'B0BBM919DP',
 'B0BYSY8NXV',
 'B09F76BCZS',
 'B0BQ6P583M',
 'B0B62LR2YT',
 'B09YTGFYZK',
 'B09Z4R6Y8S',
 'B0B5HMT1YW',
 'B09STGSRW2',
 'B0BH7JYSPX',
 'B09R1TMXR3',
 'B0C3CL7VP4',
 'B0BF9DQB6K',
 'B0BJNTLJ5X',
 'B0BK14FQMQ',
 'B0B46D4FM3',
 'B0B6Z2978H',
 'B09RJP7YM2',
 'B0BD4RWZLZ',
 'B09LVX3XW2',
 'B0BCKCJQPN',
 'B0B6ZZH83Y',
 'B0BVZKWFXB',
 'B09VR5N1FN',
 'B0C6QS154C',
 'B0BQ6PVB8S',
 'B09QS7W8G5',
 'B0B2W9G1M8',
 'B0BR8Q2PXN',
 'B0C4Y1JRRB',
 'B09WVHZ4X8',
 'B0C6YFW3G6',
 'B0BBWLP9KV',
 'B0BJ1MG9JL',
 'B09TDJBZ7X',
 'B0CG68ZMFV',
 'B0C7Q3X76Q',
 'B09SFN9NRX',
 'B0B8S4D469',
 'B0CBBBTY2C',
 'B0BY1ZYQPQ',
 'B0BDZY67PK',
 'B08WQ55H4Y',
 'B09YD1FZDN',
 'B0B5H7T7XZ',
 'B09T2J1LSB',
 'B0B8CKD3TN',
 'B09ZTX4J72',
 'B0C9T5H9F2',
 'B096G85CLH',
 'B09R6STLP7',
 'B0BTYQ7VVM',
 'B09TKFDCV5',
 'B0BYYGZHG5',
 'B0BW89VYWN',
 'B0C74NFF6F',
 'B0BML99MQ3',
 'B0BQHNB35H',
 'B0C5CLD1HF',
 'B0B4B1XPN3',
 'B0BJZV7QVF',
 'B0C378ZL2K',
 'B0B11P5X

In [9]:
len(parent_asin_list)

1000

### Generate Synthetic availability for all items in Qdrant

In [10]:
def generate_inventory_data(warehouses, product_ids, availability_rate=0.75):
    
    inventory_records = []
    
    for warehouse in warehouses:
        for product_id in product_ids:
            # 75% chance the product is available in this warehouse
            if random.random() < availability_rate:
                total_quantity = random.randint(0, 100)
                
                # Only add to inventory if quantity > 0
                if total_quantity > 0:
                    inventory_records.append({
                        "warehouse_id": warehouse["warehouse_id"],
                        "warehouse_location": warehouse["warehouse_location"],
                        "warehouse_name": warehouse["warehouse_name"],
                        "product_id": product_id,
                        "total_quantity": total_quantity,
                        "reserved_quantity": 0  # Starting with no reservations
                    })
    
    return inventory_records

In [11]:
inventory_data = generate_inventory_data(warehouses, parent_asin_list, availability_rate=0.75)

In [12]:
inventory_data

[{'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0C2YMY2QB',
  'total_quantity': 89,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B09PYN3P4B',
  'total_quantity': 33,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0BM79FR3B',
  'total_quantity': 92,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0BN54T6XD',
  'total_quantity': 42,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0BBM919DP',
  'total_quantity': 78,
  

In [13]:
1000*0.75*6

4500.0

In [14]:
len(inventory_data)

4462

### Write syntehetic data to Postgres

In [15]:
def insert_inventory_to_db(inventory_records):
   
    try:
        # Connect to the database
        conn = psycopg2.connect(
            host="localhost",
            port=5433,
            database="tools_database",
            user="langraph_user",
            password="langraph_password"
        )
        conn.autocommit = True

        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        
            # Prepare the INSERT query
            insert_query = """
            INSERT INTO warehouses.inventory 
            (warehouse_id, warehouse_location, warehouse_name, product_id, total_quantity, reserved_quantity)
            VALUES (%(warehouse_id)s, %(warehouse_location)s, %(warehouse_name)s, %(product_id)s, %(total_quantity)s, %(reserved_quantity)s)
            """
            
            # Use execute_batch for better performance with many inserts
            execute_batch(cursor, insert_query, inventory_records, page_size=100)
            
            # Commit the transaction
            conn.commit()
            
            print(f"Successfully inserted {len(inventory_records)} records into warehouses.inventory")
            
            # Close cursor and connection
            cursor.close()
            conn.close()
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [16]:
insert_inventory_to_db(inventory_data)

Successfully inserted 4462 records into warehouses.inventory
